In [1]:
from aind_data_access_api.document_db import MetadataDbClient
import os
import pandas as pd

In [2]:
DOC_DB_HOST = os.getenv("CUSTOM_KEY_3")
DOC_DB_DATABASE = "metadata"
DOC_DB_COLLECTION = "data_assets"

doc_db_client = MetadataDbClient(host=DOC_DB_HOST, database=DOC_DB_DATABASE, collection=DOC_DB_COLLECTION,)
results = doc_db_client.retrieve_data_asset_records(filter_query=None, projection={"_name":1, "_created": 1, "_location":1, "data_description.data_level": 1}, paginate=False)

In [3]:
print(results[5])

has_data_desc = []


data_levels = {}
for result in results:
    try:
        if result.data_description['data_level'] not in data_levels:
            data_levels[result.data_description['data_level']] = 1
        else:
            data_levels[result.data_description['data_level']] += 1
            
        has_data_desc.append(result)
    except:
        continue
            
data_levels

id='1665b876-02c7-4463-ba0f-377def1e7ca7' name='SmartSPIM_699318_2023-12-07_21-41-26' created=datetime.datetime(2023, 12, 8, 13, 33, 2, tzinfo=TzInfo(UTC)) location='s3://aind-open-data/SmartSPIM_699318_2023-12-07_21-41-26' data_description={'data_level': 'raw data'}


{'raw': 613, 'raw data': 3361, 'derived data': 1042, 'derived': 289}

In [4]:
old_tags = [result._name for result in has_data_desc if result.data_description['data_level'] == 'raw' or result.data_description['data_level'] == 'derived']
print(old_tags)

['ecephys_687360_2023-12-06_08-54-08', 'ecephys_687357_2023-11-06_17-13-59', 'ecephys_681532_2023-10-16_12-44-24_sorted-ks2.5_2023-10-26_10-37-17', 'ecephys_703545_2023-11-08_18-46-14_sorted_2023-11-09_08-51-44', 'ecephys_703555_2023-11-08_19-44-28_sorted_2023-11-09_08-34-38', 'ecephys_622466_2022-07-18_10-38-29_sorted_2023-10-30_14-06-34', 'ecephys_623708_2022-07-22_10-37-21_sorted-ks2.5_2023-10-30_14-33-59', 'ecephys_623705_2022-06-27_16-57-47_sorted-ks2.5_2023-10-30_14-22-03', 'multiplane-ophys_644542_2022-11-03_15-38-29', 'multiplane-ophys_645814_2022-11-11_12-34-11', 'multiplane-ophys_687001_2023-10-20_11-40-31', 'multiplane-ophys_687000_2023-10-18_11-31-27', 'ecephys_702459_2023-12-20_12-44-03_sorted_2023-12-21_02-11-38', 'ecephys_702459_2023-12-20_14-45-03_sorted_2023-12-21_01-41-47', 'ecephys_702459_2023-12-20_10-51-51', 'ecephys_702459_2023-12-20_12-44-03', 'ecephys_702459_2023-12-20_14-05-18', 'ecephys_703554_2023-12-20_15-46-44', 'ecephys_702459_2023-12-20_11-34-54', 'ecephy

In [5]:
whodunnit = set([result.split('_')[0] for result in old_tags])
print(whodunnit)

{'behavior', 'confocal', 'exaSPIM', 'multiplane-ophys', 'ecephys', 'single-plane-ophys'}


In [7]:
df = pd.DataFrame(columns=['asset_name','tags'])
for result in has_data_desc:
    if result.data_description['data_level'] == 'raw' or result.data_description['data_level'] == 'derived':
        df = pd.concat([pd.DataFrame([[result._name, result.data_description['data_level']]], columns=df.columns), df], ignore_index=True)
    
df

,asset_name,tags
0,ecephys_625463_2022-09-30_14-37-42,raw
1,multiplane-ophys_612764_2022-03-28_12-21-23,raw
2,exaSPIM_654306_2023-10-09_14-44-13,raw
3,multiplane-ophys_645814_2022-10-26_16-50-48,raw
4,multiplane-ophys_697256_2023-11-17_12-26-33,raw
...,...,...
897,ecephys_703555_2023-11-08_19-44-28_sorted_2023...,derived
898,ecephys_703545_2023-11-08_18-46-14_sorted_2023...,derived
899,ecephys_681532_2023-10-16_12-44-24_sorted-ks2....,derived
900,ecephys_687357_2023-11-06_17-13-59,raw


In [9]:
results_file_path = "/results/old_tags.txt"

with open(results_file_path, 'a') as f:
    df_string = df.to_string(header=False, index=False)
    f.write(df_string)